In [4]:
# DataFrame
import numpy as np
import pandas as pd

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#Visualization
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn import metrics

# Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Word2vec
import gensim

# Utility
import re
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [35]:
path = "./data/Sentiment140.tenPercent.sample.tweets.csv"
columns = ["target", "ids", "date", "flag", "user", "text"]
df = pd.read_csv(path, sep="\t", on_bad_lines='skip', encoding = "ISO-8859-1") # , header=None encoding='ISO-8859-1', names=columns
df.head()

,sentiment_label,tweet_text
0,4,"@elephantbird Hey dear, Happy Friday to You A..."
1,4,Ughhh layin downnnn Waiting for zeina to co...
2,0,"@greeniebach I reckon he'll play, even if he's..."
3,0,@vaLewee I know! Saw it on the news!
4,0,very sad that http://www.fabchannel.com/ has c...


In [36]:
df.columns

Index(['sentiment_label', 'tweet_text'], dtype='object')

In [39]:
df.isna().sum()

sentiment_label    0
tweet_text         0
dtype: int64

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Read the dataset
data = df.copy()

# Split the dataset into features (X) and target (y)
X = data['tweet_text']
y = data['sentiment_label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Initialize the Random Forest Classifier
clf = RandomForestClassifier()

# Train the model on the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Evaluate the model by checking the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
import numpy as np

# Load the data
data = [("4\t@elephantbird Hey dear, Happy Friday to You Already had your rice's bowl for lunch ?", 4),
        ("4\tUghhh layin downnnn Waiting for zeina to cook breakfast", 4),
        ("0\t@greeniebach I reckon he'll play, even if he's not 100%...but i know nothing!! ;) It won't be the same without him.", 0),
        # ... the rest of the data ...
       ]
texts, labels = zip(*data)

# Preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
texts = tokenizer.texts_to_sequences(texts)
texts = pad_sequences(texts, maxlen=100, padding="post", truncating="post")
labels = np.array(labels)

# Split the data into training and testing sets
training_data = texts[:int(len(texts)*0.8)]
training_labels = labels[:int(len(texts)*0.8)]
testing_data = texts[int(len(texts)*0.8):]
testing_labels = labels[int(len(texts)*0.8):]

# Define the CNN model
cnn_model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=100),
    Conv1D(64, 3, activation="relu"),
    GlobalMaxPool1D(),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])
cnn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Define the RNN model
rnn_model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=100),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPool1D(),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])
rnn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the models
cnn_model.fit(training_data, training_labels, epochs=10, validation_data=(testing_data, testing_labels))
rnn_model.fit(training_data, training_labels, epochs=10, validation_data=(testing_data, testing_labels))

cnn_scores = cnn_model.evaluate(testing_data, testing_labels, verbose=0)
rnn_scores = rnn_model.evaluate(testing_data, testing_labels, verbose=0)

